In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pisa.administrative_area import AdministrativeArea
from pisa.facilities import Facilities
from pisa.osm_road_network import OsmRoadNetwork
from pisa.population import FacebookPopulation, WorldpopPopulation
from pisa.visualisation import plot_facilities, plot_isochrones, plot_population, plot_population_heatmap


### Define administrative area

Let op: the administrative area is a country because of the package that we use for fetching the data (gadm)! 

The naming is confusing: we want the administrative area to be a subset of a country (in this case, the administrative area should be Baucau). 

Should be easy to fix once we change gadm to pygadm (see issue #59 on GitHub)

In [ ]:
timor_leste = AdministrativeArea(country_name="Timor-Leste", admin_level=1)

# these are the boundaries of Baucau
# type: Polygon
baucau = timor_leste.get_admin_area_boundaries("Baucau")

### Get hospitals from osm

In [ ]:
hospitals = Facilities(admin_area_boundaries=baucau).get_existing_facilities()

# todo: suppress user warning Geometry is in a geographic CRS, and
# add documentation on why this is ok

In [ ]:
hospitals.head()

In [ ]:
plot_facilities(hospitals, baucau)

In [ ]:
potential_hospitals = Facilities(admin_area_boundaries=baucau).estimate_potential_facilities(spacing=0.05)
plot_facilities(potential_hospitals, baucau)

### Get population from WorldPop

In [ ]:
# TODO: why "ID" column? extra index for what reason? Maybe becomes apparent later?

population = WorldpopPopulation(
    admin_area_boundaries=baucau, iso3_country_code=timor_leste.get_iso3_country_code()
).get_population_gdf()

population.head()

In [ ]:
plot_population_heatmap(population, baucau)

In [ ]:
plot_population(population, baucau, random_sample_n=1000)

### Alternatively: get population from facebook

In [ ]:
population_fb = FacebookPopulation(
    admin_area_boundaries=baucau, iso3_country_code=timor_leste.get_iso3_country_code()
).get_population_gdf()

population_fb.head()

### Isopolygons

Here we make some choices:
- mode of transport 
- distance type

Valid values are in the script pisa.constants

#### Option 1: using OSM

For this, we need to get the road network from osmnx

In [ ]:
road_network = OsmRoadNetwork(
    admin_area_boundaries=baucau, mode_of_transport="driving", distance_type="length", fallback_speed=20.5
).get_osm_road_network()

In [ ]:
from pisa.isopolygons import OsmIsopolygonCalculator

isopolygons = OsmIsopolygonCalculator(
    facilities_df=hospitals, distance_type="length", distance_values=[1500, 500], road_network=road_network
).calculate_isopolygons()

In [ ]:
isopolygons

In [ ]:
plot_isochrones(isopolygons, baucau)